# TPOTを用いてSIGNATEの楽曲のジャンル推定チャレンジ！に挑戦する


## ライブラリのインポート

In [58]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import tpot
from tpot import TPOTClassifier
from sklearn.metrics import confusion_matrix#混同行列(誤差行列)の表示

## 各変数の設定

In [59]:
# 学習データのラベル
train_master_path = "/Users/oonoharuki/competition/SIGNATE/SOTAchallenge/楽曲のジャンル推定チャレンジ/train.csv"
#テストデータのラベル
test_master_path = "/Users/oonoharuki/competition/SIGNATE/SOTAchallenge/楽曲のジャンル推定チャレンジ/test02.csv"
#保存作のパス
submit_master_path = "/Users/oonoharuki/competition/SIGNATE/SOTAchallenge/楽曲のジャンル推定チャレンジ/submit.csv"

## データの読み込みと前処理

In [60]:
#学習用画像データの読み込み
train = pd.read_csv(train_master_path)
train

,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region
0,0,10,11,201094,0.112811,0.157247,0.187841,-1.884852,0.893918,0.363568,0.390108,0.888884,121-152,region_H
1,1,8,69,308493,0.101333,0.346563,0.554444,-5.546495,0.874409,0.193892,0.161497,0.123910,153-176,region_I
2,2,3,43,197225,0.496420,0.265391,0.457642,-9.255670,0.439933,0.217146,0.369057,0.166470,64-76,region_E
3,3,10,45,301092,0.165667,0.245533,0.356578,-5.088788,0.868704,0.377025,0.226677,0.175399,177-192,region_C
4,4,3,57,277348,0.190720,0.777578,0.830479,-3.933896,0.650149,0.169323,0.222488,0.226030,97-120,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4041,4041,10,38,246309,0.128795,0.329718,0.575830,-5.252543,0.509531,0.193781,0.187331,0.158197,121-152,region_P
4042,4042,5,23,208734,0.778732,0.228757,0.394283,-14.225700,0.322871,0.253108,0.141218,0.295608,121-152,region_D
4043,4043,10,30,407016,0.402050,0.462374,0.672265,-10.711253,0.646571,0.228189,0.152270,0.163483,97-120,region_E
4044,4044,10,25,204957,0.208096,0.465511,0.523514,-7.855946,0.508910,0.247820,0.202238,0.210184,77-96,region_R


In [61]:
#index部分の削除
train = train.iloc[:,1:]
train

,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region
0,10,11,201094,0.112811,0.157247,0.187841,-1.884852,0.893918,0.363568,0.390108,0.888884,121-152,region_H
1,8,69,308493,0.101333,0.346563,0.554444,-5.546495,0.874409,0.193892,0.161497,0.123910,153-176,region_I
2,3,43,197225,0.496420,0.265391,0.457642,-9.255670,0.439933,0.217146,0.369057,0.166470,64-76,region_E
3,10,45,301092,0.165667,0.245533,0.356578,-5.088788,0.868704,0.377025,0.226677,0.175399,177-192,region_C
4,3,57,277348,0.190720,0.777578,0.830479,-3.933896,0.650149,0.169323,0.222488,0.226030,97-120,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4041,10,38,246309,0.128795,0.329718,0.575830,-5.252543,0.509531,0.193781,0.187331,0.158197,121-152,region_P
4042,5,23,208734,0.778732,0.228757,0.394283,-14.225700,0.322871,0.253108,0.141218,0.295608,121-152,region_D
4043,10,30,407016,0.402050,0.462374,0.672265,-10.711253,0.646571,0.228189,0.152270,0.163483,97-120,region_E
4044,10,25,204957,0.208096,0.465511,0.523514,-7.855946,0.508910,0.247820,0.202238,0.210184,77-96,region_R


In [62]:
#重複しているカラムがあるのかの確認
train.drop_duplicates()

,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region
0,10,11,201094,0.112811,0.157247,0.187841,-1.884852,0.893918,0.363568,0.390108,0.888884,121-152,region_H
1,8,69,308493,0.101333,0.346563,0.554444,-5.546495,0.874409,0.193892,0.161497,0.123910,153-176,region_I
2,3,43,197225,0.496420,0.265391,0.457642,-9.255670,0.439933,0.217146,0.369057,0.166470,64-76,region_E
3,10,45,301092,0.165667,0.245533,0.356578,-5.088788,0.868704,0.377025,0.226677,0.175399,177-192,region_C
4,3,57,277348,0.190720,0.777578,0.830479,-3.933896,0.650149,0.169323,0.222488,0.226030,97-120,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4041,10,38,246309,0.128795,0.329718,0.575830,-5.252543,0.509531,0.193781,0.187331,0.158197,121-152,region_P
4042,5,23,208734,0.778732,0.228757,0.394283,-14.225700,0.322871,0.253108,0.141218,0.295608,121-152,region_D
4043,10,30,407016,0.402050,0.462374,0.672265,-10.711253,0.646571,0.228189,0.152270,0.163483,97-120,region_E
4044,10,25,204957,0.208096,0.465511,0.523514,-7.855946,0.508910,0.247820,0.202238,0.210184,77-96,region_R


In [63]:
#train内の情報の確認
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4046 entries, 0 to 4045
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   genre             4046 non-null   int64  
 1   popularity        4046 non-null   int64  
 2   duration_ms       4046 non-null   int64  
 3   acousticness      4046 non-null   float64
 4   positiveness      4036 non-null   float64
 5   danceability      4038 non-null   float64
 6   loudness          4046 non-null   float64
 7   energy            4046 non-null   float64
 8   liveness          4043 non-null   float64
 9   speechiness       4038 non-null   float64
 10  instrumentalness  4045 non-null   float64
 11  tempo             4046 non-null   object 
 12  region            4046 non-null   object 
dtypes: float64(8), int64(3), object(2)
memory usage: 411.0+ KB


In [64]:
#Dtypeを揃えるため、カテゴリ変数をダミー変数に変換
train = pd.get_dummies(train,drop_first=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4046 entries, 0 to 4045
Data columns (total 42 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   genre             4046 non-null   int64  
 1   popularity        4046 non-null   int64  
 2   duration_ms       4046 non-null   int64  
 3   acousticness      4046 non-null   float64
 4   positiveness      4036 non-null   float64
 5   danceability      4038 non-null   float64
 6   loudness          4046 non-null   float64
 7   energy            4046 non-null   float64
 8   liveness          4043 non-null   float64
 9   speechiness       4038 non-null   float64
 10  instrumentalness  4045 non-null   float64
 11  tempo_121-152     4046 non-null   uint8  
 12  tempo_153-176     4046 non-null   uint8  
 13  tempo_177-192     4046 non-null   uint8  
 14  tempo_193-208     4046 non-null   uint8  
 15  tempo_209-220     4046 non-null   uint8  
 16  tempo_41-50       4046 non-null   uint8  


In [65]:
#今回の目的変数であるgenreの除去
x = train.drop("genre",axis = 1)
x

,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,...,region_region_L,region_region_M,region_region_N,region_region_O,region_region_P,region_region_Q,region_region_R,region_region_S,region_region_T,region_unknown
0,11,201094,0.112811,0.157247,0.187841,-1.884852,0.893918,0.363568,0.390108,0.888884,...,0,0,0,0,0,0,0,0,0,0
1,69,308493,0.101333,0.346563,0.554444,-5.546495,0.874409,0.193892,0.161497,0.123910,...,0,0,0,0,0,0,0,0,0,0
2,43,197225,0.496420,0.265391,0.457642,-9.255670,0.439933,0.217146,0.369057,0.166470,...,0,0,0,0,0,0,0,0,0,0
3,45,301092,0.165667,0.245533,0.356578,-5.088788,0.868704,0.377025,0.226677,0.175399,...,0,0,0,0,0,0,0,0,0,0
4,57,277348,0.190720,0.777578,0.830479,-3.933896,0.650149,0.169323,0.222488,0.226030,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4041,38,246309,0.128795,0.329718,0.575830,-5.252543,0.509531,0.193781,0.187331,0.158197,...,0,0,0,0,1,0,0,0,0,0
4042,23,208734,0.778732,0.228757,0.394283,-14.225700,0.322871,0.253108,0.141218,0.295608,...,0,0,0,0,0,0,0,0,0,0
4043,30,407016,0.402050,0.462374,0.672265,-10.711253,0.646571,0.228189,0.152270,0.163483,...,0,0,0,0,0,0,0,0,0,0
4044,25,204957,0.208096,0.465511,0.523514,-7.855946,0.508910,0.247820,0.202238,0.210184,...,0,0,0,0,0,0,1,0,0,0


In [66]:
#目的変数genreを設定する
y = train["genre"]

In [67]:
#train_test_splitを用いて訓練用とテスト用に分割する
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.6,random_state=0,stratify=y)

## 学習モデルの作成
[こちら](http://epistasislab.github.io/tpot/api/)を参照

In [68]:
tpot = TPOTClassifier(generations=100,#世代数。パイプラインの最適化を実行する試行回数に相当。
                      population_size=100,#世代ごとに保持するパイプライン数。
                      offspring_size =None,#子孫数。世代ごとに幾つのパイプラインが生成されるかに相当する
                      #population_size+generations*offspring_sizeの数だけパイプラインが生成されます。
                      
                      mutation_rate=0.9,#遺伝的プログラミングアルゴリズムの変異率
                      crossover_rate=0.1,#遺伝的プログラミングアルゴリズムのクロスオーバー率
                      #mutation_rate+crossover_rateの合計が1になるようにする。
                      
                      scoring=None,#作成されたパイプラインの評価指標
                      cv = 5,#交差検証を指す。これは個々のモデルの汎用性能を評価するもので今回は5個に分けて学習を行なっていることを指す
                      subsample=0.8,#トレーニングデータをどのくらい用いるのか。1.0で全て用いる1.0未満で過学習を抑えることができるかもしれない
                      n_jobs=-1,#使用されるCPUの数-1にすると全てのcpuが使用される
                      max_time_mins=None,#tpotが最適化を行う最大分数
                      max_eval_time_mins=10,#TPOTが単一のパイプラインを評価するために必要な分数。
                      random_state=None,#毎回ランダムにするのかどうか
                      config_dict=None,#TPOTが最適化プロセスで検索する演算子とパラメーターをカスタマイズ。TPOT light,TPOT MDR,TPOT sparse,Noneの入力が可能
                      template=None,#事前定義されたパイプライン構造のテンプレート。
                      warm_start=True,
                      memory=None,
                      use_dask=False,
                      periodic_checkpoint_folder=None,
                      early_stop=None,
                      verbosity=3,
                      disable_update_check=False,
                      log_file=None,
                     )
tpot.fit(x_train,y_train)#学習の実行
print("Accuracy:",tpot.score(x_test,y_test))
print(confusion_matrix(y_test,tpot.predict(x_test)))
print("Best Pipeline:",tpot.fitted_pipeline_)#最も評価の高いパイプラインの表示

32 operators have been imported by TPOT.
Imputing missing values in feature set
Optimization Progress:   0%|          | 0/10100 [00:00<?, ?pipeline/s]

/Users/oonoharuki/.pyenv/versions/3.9.4/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/oonoharuki/.pyenv/versions/3.9.4/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/oonoharuki/.pyenv/versions/3.9.4/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/oonoharuki/.pyenv/versions/3.9.4/lib/python3.9/site-packages/tpot/builtins/__init

_pre_test decorator: _random_mutation_operator: num_test=0 Expected n_neighbors <= n_samples,  but n_samples = 50, n_neighbors = 81.
_pre_test decorator: _random_mutation_operator: num_test=0 Negative values in data passed to MultinomialNB (input X).
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l1' and loss='hinge' is not supported, Parameters: penalty='l1', loss='hinge', dual=True.
Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.
                                                                                   
Generation 1 - Current Pareto front scores:
                                                                                   
-1	0.6481210081361143	ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=False, ExtraTreesClassifier__criterion=entropy, ExtraTreesClassifier__max_features=0.55, ExtraTreesClassif

/Users/oonoharuki/.pyenv/versions/3.9.4/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/oonoharuki/.pyenv/versions/3.9.4/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


                                                                                   
Generation 2 - Current Pareto front scores:
                                                                                   
-1	0.6481210081361143	ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=False, ExtraTreesClassifier__criterion=entropy, ExtraTreesClassifier__max_features=0.55, ExtraTreesClassifier__min_samples_leaf=3, ExtraTreesClassifier__min_samples_split=5, ExtraTreesClassifier__n_estimators=100)
                                                                                   
-2	0.6682347017199799	GradientBoostingClassifier(ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=False, ExtraTreesClassifier__criterion=gini, ExtraTreesClassifier__max_features=0.5, ExtraTreesClassifier__min_samples_leaf=1, ExtraTreesClassifier__min_samples_split=12, ExtraTreesClassifier__n_estimators=100), GradientBoostingClassifier__learning_rate=0.1, GradientBoostingClassifier

...

                                                                                      
Generation 23 - Current Pareto front scores:
                                                                                      
-1	0.6846990697797685	ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=False, ExtraTreesClassifier__criterion=gini, ExtraTreesClassifier__max_features=1.0, ExtraTreesClassifier__min_samples_leaf=1, ExtraTreesClassifier__min_samples_split=6, ExtraTreesClassifier__n_estimators=100)
                                                                                      
-2	0.6877878779847879	ExtraTreesClassifier(ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=False, ExtraTreesClassifier__criterion=gini, ExtraTreesClassifier__max_features=0.7500000000000001, ExtraTreesClassifier__min_samples_leaf=1, ExtraTreesClassifier__min_samples_split=5, ExtraTreesClassifier__n_estimators=100), ExtraTreesClassifier__bootstrap=False, ExtraTreesClassifier

/Users/oonoharuki/.pyenv/versions/3.9.4/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/oonoharuki/.pyenv/versions/3.9.4/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/oonoharuki/.pyenv/versions/3.9.4/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


                                                                                       
Generation 30 - Current Pareto front scores:
                                                                                       
-1	0.6857445737153156	ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=False, ExtraTreesClassifier__criterion=gini, ExtraTreesClassifier__max_features=0.7000000000000001, ExtraTreesClassifier__min_samples_leaf=1, ExtraTreesClassifier__min_samples_split=5, ExtraTreesClassifier__n_estimators=100)
                                                                                       
-2	0.6877998038851934	ExtraTreesClassifier(ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=False, ExtraTreesClassifier__criterion=gini, ExtraTreesClassifier__max_features=0.7500000000000001, ExtraTreesClassifier__min_samples_leaf=1, ExtraTreesClassifier__min_samples_split=4, ExtraTreesClassifier__n_estimators=100), ExtraTreesClassifier__bootstrap=False, Ex

/Users/oonoharuki/.pyenv/versions/3.9.4/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/oonoharuki/.pyenv/versions/3.9.4/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/oonoharuki/.pyenv/versions/3.9.4/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


                                                                                       
                                                                                       
TPOT closed during evaluation in one generation.
                                                                                       
                                                                                       
TPOT closed prematurely. Will use the current best pipeline.


## 評価

In [ ]:
#テスト用データの読み込み
test = pd.read_csv(test_master_path)

In [ ]:
test = test.iloc[:,1:]

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4047 entries, 0 to 4046
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   popularity        4047 non-null   int64  
 1   duration_ms       4047 non-null   int64  
 2   acousticness      4046 non-null   float64
 3   positiveness      4033 non-null   float64
 4   danceability      4036 non-null   float64
 5   loudness          4047 non-null   float64
 6   energy            4046 non-null   float64
 7   liveness          4041 non-null   float64
 8   speechiness       4036 non-null   float64
 9   instrumentalness  4045 non-null   float64
 10  tempo             4047 non-null   object 
 11  region            4047 non-null   object 
dtypes: float64(8), int64(2), object(2)
memory usage: 379.5+ KB


In [ ]:
test = pd.get_dummies(test,drop_first=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4047 entries, 0 to 4046
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   popularity        4047 non-null   int64  
 1   duration_ms       4047 non-null   int64  
 2   acousticness      4046 non-null   float64
 3   positiveness      4033 non-null   float64
 4   danceability      4036 non-null   float64
 5   loudness          4047 non-null   float64
 6   energy            4046 non-null   float64
 7   liveness          4041 non-null   float64
 8   speechiness       4036 non-null   float64
 9   instrumentalness  4045 non-null   float64
 10  tempo_121-152     4047 non-null   uint8  
 11  tempo_153-176     4047 non-null   uint8  
 12  tempo_177-192     4047 non-null   uint8  
 13  tempo_193-208     4047 non-null   uint8  
 14  tempo_209-220     4047 non-null   uint8  
 15  tempo_41-50       4047 non-null   uint8  
 16  tempo_51-56       4047 non-null   uint8  


In [ ]:
#検証用データの予測
pred = tpot.predict(test)

Imputing missing values in feature set


In [ ]:
#予測データをデータフレームに格納
pred = pd.DataFrame(pred,columns=["genre"])

In [ ]:
#idを取得するためにテスト用データを再度読み込み
test_id = pd.read_csv(test_master_path)

In [ ]:
#o番目にあるindexを持ってくる
test_id  = test_id.iloc[:,0]

In [ ]:
#先ほどのtest_idとpredを結合する
submit = pd.concat([test_id,pred],axis = 1)

In [ ]:
submit

,index,genre
0,4046,7
1,4047,10
2,4048,8
3,4049,8
4,4050,8
...,...,...
4042,8088,8
4043,8089,1
4044,8090,8
4045,8091,7


In [ ]:
#csvファイルに格納する
submit.to_csv(submit_master_path,header=None,index=None)